In [ ]:
# Correct the import statement
from IPython.display import display, Image, clear_output
import time
import requests
from bs4 import BeautifulSoup
import json


# Introduction
print("Welcome to the Memory test")
time.sleep(1)

user_name = input("Please enter your name: ").strip().title()  
print(f"Hello, {user_name}! Let's get started with the test.")
time.sleep(1)

id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:

- two letters based on the initials (first and last name) of a childhood friend

- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise

then your unique identifer would be CBTC

"""

print(id_instructions)

user_id = input("> ")

print("User entered id:", user_id)
time.sleep(2)

print("Please enter your gender: M or F")
gender = input(">")
print("Gender:",gender)
time.sleep(2)

print("There are three small tests in this section, getting harder each time")
time.sleep(2)

print("Each test consists of a grid of coloured shapes for 30 seconds. You should memorize them as quickly as possible and answer some questions")
time.sleep(2)

print("Good luck to you. Here comes the first warm-up question")
time.sleep(2)


# Ask questions and conduct tests
def ask_question(question, correct_answer):
    user_answer = input(question + " ").strip().lower()
    if user_answer == correct_answer.lower():
        print("Correct!")
        return 1
    else:
        print(f"Incorrect. The correct answer is: {correct_answer}")
        return 0

def conduct_test(test_image, questions_and_answers):
    display(Image(test_image, width=300))
    time.sleep(30)  
    clear_output(wait=False)
    score = 0
    for question, correct_answer in questions_and_answers.items():
        score += ask_question(question, correct_answer)
    return score

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

questions_test1 = {
    "What is the color of the circle?": 'yellow',
    "What shape was above the heart?": "square",
    "What shape was between the star and the moon?": "heart",
    "What shape was below the triangle?": "Star"
}

questions_test2 = {
    "What is the color of the moon?": 'red',
    "What is the shape in the top right corner?": 'sun',
    "How many shapes are yellow?": '4',
    "What is the color of the star?":'green'
}

questions_test3 = {
    "What is the color of the cloud?": 'yellow',
    "Which direction is the arrow pointing?": 'right',
    "What is the shape between the circle and the sun?": 'pentagon',
    "What is the shape in the bottom left corner?":'sun'
}


# Calculate the total score and percentage
score_test1 = conduct_test("memory_test1.png", questions_test1)
print(f"Your score for test 1 is: {score_test1} out of {len(questions_test1)}")

score_test2 = conduct_test("memory_test2.png", questions_test2)
print(f"Your score for test 2 is: {score_test2} out of {len(questions_test2)}")

score_test3 = conduct_test("memory_test3.png", questions_test3)
print(f"Your score for test 3 is: {score_test3} out of {len(questions_test3)}")

total_score = score_test1 + score_test2 + score_test3
total_questions = len(questions_test1) + len(questions_test2) + len(questions_test3)
total_percentage = (total_score / total_questions) * 100
print(f"You've got a total of {total_percentage}% in memory test")

# Calculate accuracy for each section
accuracy_test1 = score_test1 / len(questions_test1)
accuracy_test2 = score_test2 / len(questions_test2)
accuracy_test3 = score_test3 / len(questions_test3)

#Save scores in google form
print("Please read:")
print("")
print("we wish to record your response data")
print("to an anonymised public data repository. ")
print("Your data will be used for educational teaching purposes")
print("practising data analysis and visualisation.")
print("")
print("Please type   yes   in the box below if you consent to the upload.")
result = input("> ")
if result == "yes":
    print("Thanks - your data will be uploaded.")
    form_data = {
        'Username': user_name,
        'User_id': user_id,
        'Gender': gender,
        'Section_1_Accuracy': accuracy_test1, 
        'Section_2_Accuracy': accuracy_test2,  
        'Section_3_Accuracy': accuracy_test3,  
        'Total_Percentage': total_percentage  
    }
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSfSn_A_LngagdlDxMOfHcdjNM9LKDcDyg70LMh3YkDUiVlAKQ/formResponse'
    send_to_google_form(form_data, form_url)
else:
    print("No problem we hope you enjoyed the test.")